In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ansh1601","key":"89a23d3b9d9732d82169bc93bd28134c"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c deep-learning-for-msc-2022-23

 93% 208M/224M [00:00<00:00, 243MB/s]
100% 224M/224M [00:01<00:00, 229MB/s]


In [4]:
!unzip deep-learning-for-msc-2022-23.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/3302.png          
  inflating: train/3303.png          
  inflating: train/3304.png          
  inflating: train/3305.png          
  inflating: train/3306.png          
  inflating: train/3307.png          
  inflating: train/3308.png          
  inflating: train/3309.png          
  inflating: train/331.png           
  inflating: train/3310.png          
  inflating: train/3311.png          
  inflating: train/3312.png          
  inflating: train/3313.png          
  inflating: train/3314.png          
  inflating: train/3315.png          
  inflating: train/3316.png          
  inflating: train/3317.png          
  inflating: train/3318.png          
  inflating: train/3319.png          
  inflating: train/332.png           
  inflating: train/3320.png          
  inflating: train/3321.png          
  inflating: train/3322.png          
  inflating: train/3323.png          
  inflating: train/3324.png          

In [5]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('example.csv')

In [6]:
import torch
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.df.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.df.iloc[idx, 1]
        return image, label

# Define image transformations
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.RandomPerspective(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load custom train dataset using the defined class
train_dataset = CustomDataset('train.csv', 'train', transform=transform)

# Load custom test dataset using the defined class
test_dataset = CustomDataset('example.csv', 'test', transform=transform)

# Create DataLoader for batching and shuffling


In [8]:
train_dataset[0]

(tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]), 0)

In [10]:
from torch.utils.data import random_split
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=32, shuffle=False)

In [10]:
import collections
import torch.nn as nn
'''model = nn.Sequential(collections.OrderedDict([
          ('conv1', nn.Conv2d(3,1024,3,padding=1)),
          ('batch1',nn.Batchnorm2d(1024)),
          ('relu1', nn.ReLU()),

          ('conv2', nn.Conv2d(1024,512,3,padding=1)),
          ('batch2',nn.Batchnorm2d(512)),
          ('relu2', nn.ReLU()),
          ('conv3', nn.Conv2d(512,256,3,padding=1)),
          ('batch3',nn.Batchnorm2d(256)),
          ('relu3', nn.ReLU()),
          ('conv4', nn.Conv2d(256,128,3,padding=1)),
          ('batch4',nn.Batchnorm2d(128)),
          ('relu4', nn.ReLU()),
          ('conv5', nn.Conv2d(128,64,3,padding=1)),
          ('batch5',nn.Batchnorm2d(64))
          ('relu5', nn.ReLU()),

          # Put in a linear layers ...
          ('flatten', nn.Flatten()),                                    
          ('fc1', nn.Linear(64*100*100,1024)),
          ('relu6', nn.ReLU()),
          ('fc2', nn.Linear(1024,4)),
          nn.Dropout(0.5),
        ]))'''
model = nn.Sequential(collections.OrderedDict([
    ('conv1', nn.Conv2d(3, 32, kernel_size=3, padding=1)),
    ('batch1', nn.BatchNorm2d(32)),
    ('relu1', nn.ReLU()),
    #('maxpool1', nn.MaxPool2d(kernel_size=2)),

    ('conv2', nn.Conv2d(32, 64, kernel_size=3, padding=1)),
    ('batch2', nn.BatchNorm2d(64)),
    ('relu2', nn.ReLU()),
    #('maxpool2', nn.MaxPool2d(kernel_size=2)),

    ('conv3', nn.Conv2d(64, 128, kernel_size=3, padding=1)),
    ('batch3', nn.BatchNorm2d(128)),
    ('relu3', nn.ReLU()),
    #('maxpool3', nn.MaxPool2d(kernel_size=2)),

    ('conv4', nn.Conv2d(128, 256, kernel_size=3, padding=1)),
    ('batch4', nn.BatchNorm2d(256)),
    ('relu4', nn.ReLU()),
    #('maxpool4', nn.MaxPool2d(kernel_size=2)),

     ('conv5', nn.Conv2d(256, 64, kernel_size=3, padding=1)),
    ('batch5', nn.BatchNorm2d(64)),
    ('relu5', nn.ReLU()),
    #('maxpool5', nn.MaxPool2d(kernel_size=2)),

    ('conv6', nn.Conv2d(64, 32, kernel_size=3, padding=1)),
    ('batch6', nn.BatchNorm2d(32)),
    ('relu6', nn.ReLU()),
    #('maxpool6', nn.MaxPool2d(kernel_size=2)),

    ('flatten', nn.Flatten()),
    ('fc1', nn.Linear(32 * 100 * 100, 1024)),
    ('relu7', nn.ReLU()),
    ('dropout', nn.Dropout(0.5)),
    ('fc2', nn.Linear(1024, 4))
]))


In [11]:
print("\nCNN Network Model:")
print(model)


CNN Network Model:
Sequential(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU()
  (conv5): Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu5): ReLU()
  (conv6): Conv2d(64, 32, ke

In [12]:
import datetime
epoch_print_gap = 1
def training_loop(n_epochs, optimizer, model, device, loss_fn, train_loader,test_loader):
    model = model.to(device)
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            outputs = model(imgs.to(device))
            loss = loss_fn(outputs, labels.to(device))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
            
        if epoch == 1 or epoch % epoch_print_gap == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch, float(loss_train)))
        model.eval()
        model = model.to(device)
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)

        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

def test_loop(model, device, test_loader):
    model.eval()
    model = model.to(device)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:
optimizer2 = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001, betas=(0.9, 0.999))

In [14]:
import torch.nn.functional as F
import torch.optim as optim
n_epochs = 10
optimizer = optim.SGD(model.parameters(), lr=0.001,momentum = 0.9)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = n_epochs, 
    optimizer = optimizer2,
    model = model,
    loss_fn = loss_fn,
    device= device,
    train_loader = train_dataloader,
    test_loader = val_dataloader
)

KeyboardInterrupt: ignored

In [ ]:
test_loop(model = model, device = device, test_loader = val_dataloader)

In [15]:
import torch
import torchvision
import torchvision.transforms as transforms


# Define the transforms for the train and test datasets
train_transform = transforms.Compose([
    
    transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


test_transform = transforms.Compose([
    
    transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the train and test datasets
train_dataset = CustomDataset('train.csv', 'train', transform=train_transform)
train_dataloader2 = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load the pre-trained ResNet18 model
model = torchvision.models.resnet18(pretrained=True)

# Replace the last layer of the model to fit the number of classes in your dataset
num_classes = 4
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, betas=(0.9,0.999),weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()
for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_dataloader2):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

# Evaluate the model on the test dataset and store the predicted labels in a list



/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[1,   100] loss: 0.407
[1,   200] loss: 0.196
[2,   100] loss: 0.081
[2,   200] loss: 0.083
[3,   100] loss: 0.048
[3,   200] loss: 0.042
[4,   100] loss: 0.039
[4,   200] loss: 0.035
[5,   100] loss: 0.022
[5,   200] loss: 0.028
[6,   100] loss: 0.017
[6,   200] loss: 0.026
[7,   100] loss: 0.015
[7,   200] loss: 0.018
[8,   100] loss: 0.009
[8,   200] loss: 0.014
[9,   200] loss: 0.018
[10,   100] loss: 0.019
[10,   200] loss: 0.019


In [16]:
test_loop(model = model, device = device, test_loader = val_dataloader)

NameError: ignored

In [17]:
from torchvision import datasets
class CustomtestDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.df.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)
        return image, img_name

test_dataset = CustomtestDataset('example.csv','test',test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Make predictions on test dataset
model.eval()

with torch.no_grad():
    predicted_labels = []
    f_names = []
    for inputs,names in test_dataloader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        predicted_labels += preds.tolist()
        f_names+=[i.split('/')[-1] for i in names]

In [18]:
import numpy as np
np.unique(predicted_labels)

array([0, 1, 2, 3])

In [19]:
import pandas as pd
pretrained=pd.read_csv('example.csv')
pretrained['Filename'] = f_names
pretrained['Label'] = predicted_labels

In [20]:
pretrained.to_csv('pretrained.csv')

In [8]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 43.8 MB/s eta 0:00:00


In [14]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split

# Define your training function
def train_model(config):
    model = torchvision.models.resnet18(pretrained=True)
    train_transform = transforms.Compose([
    
    transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])


    test_transform = transforms.Compose([
        
        transforms.RandomHorizontalFlip(),
        #transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

# Replace the last layer of the model to fit the number of classes in your dataset
    num_classes = 4
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"], betas = (0.9,0.999),weight_decay=config["weight_decay"])
    criterion = torch.nn.CrossEntropyLoss()
    train_dataset2 = CustomDataset('/content/train.csv', '/content/train', transform=train_transform)
    train_size = int(0.8 * len(train_dataset2))
    val_size = len(train_dataset2) - train_size
    train_data, val_data = random_split(train_dataset2, [train_size, val_size])
    train_loader = DataLoader(train_data, batch_size=config["batch_size"], shuffle=True)
    val_loader = DataLoader(val_data, batch_size=config["batch_size"], shuffle=False)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()
    for epoch in range(20):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 batches
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0
        
        # Evaluate on validation set
        with torch.no_grad():
            total_loss = 0
            total_correct = 0
            total_samples = 0
            for inputs, targets in val_loader:
                inputs,targets = inputs.to(device),targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                total_loss += loss.item() * inputs.size(0)
                total_correct += (outputs.argmax(1) == targets).sum().item()
                total_samples += inputs.size(0)
            val_loss = total_loss / total_samples
            val_acc = total_correct / total_samples

        # Report the validation metric for this epoch
        tune.report(val_loss=val_loss, val_acc=val_acc)
    
    # Return the final validation metric
    return {"val_loss": val_loss, "val_acc": val_acc}

# Set up the configuration dictionary
config = {
    "lr": tune.choice([1e-2, 1e-3,1e-4]),
    "weight_decay": tune.choice([1e-4, 1e-1,1e-2,1e-3]),
    "batch_size": tune.choice([16,32, 64]),
    "num_epochs": 10
}

# Set up the Ray Tune scheduler and reporter
scheduler = ASHAScheduler(
    metric="val_acc",
    mode="max",
    max_t=10,
    grace_period=1,
    reduction_factor=2
)
reporter = CLIReporter(metric_columns=["val_loss", "val_acc", "training_iteration"])

# Start the Ray Tune search
result = tune.run(
    train_model,
    resources_per_trial={"cpu": 2,"gpu":1},
    config=config,
    num_samples=10,
    scheduler=scheduler,
    progress_reporter=reporter
)

# Print the best hyperparameters found
best_trial = result.get_best_trial("val_acc", "max", "last")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(best_trial.last_result["val_loss"]))
print("Best trial final validation accuracy: {}".format(best_trial.last_result["val_acc"]))

2023-03-26 09:57:59,473	WARNING callback.py:108 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


== Status ==
Current time: 2023-03-26 09:57:59 (running for 00:00:00.16)
Memory usage on this node: 1.4/12.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/2 CPUs, 1.0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.7 GiB objects
Result logdir: /root/ray_results/train_model_2023-03-26_09-57-59
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+------------------+--------------+--------+----------------+
| Trial name              | status   | loc              |   batch_size |     lr |   weight_decay |
|-------------------------+----------+------------------+--------------+--------+----------------|
| train_model_b08dd_00000 | RUNNING  | 172.28.0.12:7402 |           64 | 0.01   |         0.1    |
| train_model_b08dd_00001 | PENDING  |                  |           32 | 0.0001 |         0.0001 |
| train_model_b08dd_00002 | PENDING  |                  |           16 | 0.0

(train_model pid=7402) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(train_model pid=7402)   warnings.warn(
(train_model pid=7402) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
(train_model pid=7402)   warnings.warn(msg)


== Status ==
Current time: 2023-03-26 09:58:08 (running for 00:00:08.53)
Memory usage on this node: 4.1/12.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/2 CPUs, 1.0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.7 GiB objects
Result logdir: /root/ray_results/train_model_2023-03-26_09-57-59
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+-------------------------+----------+------------------+--------------+--------+----------------+
| Trial name              | status   | loc              |   batch_size |     lr |   weight_decay |
|-------------------------+----------+------------------+--------------+--------+----------------|
| train_model_b08dd_00000 | RUNNING  | 172.28.0.12:7402 |           64 | 0.01   |         0.1    |
| train_model_b08dd_00001 | PENDING  |                  |           32 | 0.0001 |         0.0001 |
| train_model_b08dd_00002 | PENDING  |                  |           16 | 0.0

Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,val_acc,val_loss,warmup_time
train_model_b08dd_00000,2023-03-26_10-00-58,True,,89e17e4f4a054329b8e480dd72ee6ba8,8dd8dad0ce47,10,172.28.0.12,7402,175.374,16.8467,175.374,1679824858,0,,10,b08dd_00000,0.906746,0.253129,0.00538826
train_model_b08dd_00001,2023-03-26_10-04-09,True,,a9f2c1ca989a4b00b2b5b7ecb95b7483,8dd8dad0ce47,10,172.28.0.12,8246,186.265,17.6962,186.265,1679825049,0,,10,b08dd_00001,0.979497,0.0679228,0.00527954
train_model_b08dd_00002,2023-03-26_10-04-39,True,,5b31efdc48d94068a0778dabcafd3281,8dd8dad0ce47,1,172.28.0.12,9148,25.6816,25.6816,25.6816,1679825079,0,,1,b08dd_00002,0.755952,0.722136,0.00426817
train_model_b08dd_00003,2023-03-26_10-07-39,True,,659c3325b7a04f9a96425b1466e26112,8dd8dad0ce47,10,172.28.0.12,9334,174.295,16.1821,174.295,1679825259,0,,10,b08dd_00003,0.953704,0.175405,0.00464129
train_model_b08dd_00004,2023-03-26_10-10-27,True,,7ca2e93ec7e74af595124a71c4cf1e74,8dd8dad0ce47,10,172.28.0.12,10179,164.938,16.652,164.938,1679825427,0,,10,b08dd_00004,0.974868,0.100318,0.00503731
train_model_b08dd_00005,2023-03-26_10-10-58,True,,ddb821ddce5b4042abf9b284c88b35a5,8dd8dad0ce47,1,172.28.0.12,10972,25.6881,25.6881,25.6881,1679825458,0,,1,b08dd_00005,0.787037,0.571221,0.00611115
train_model_b08dd_00006,2023-03-26_10-11-25,True,,5a04901befd14254930c4892a74c1c71,8dd8dad0ce47,1,172.28.0.12,11157,21.7171,21.7171,21.7171,1679825485,0,,1,b08dd_00006,0.804894,0.542324,0.00676107
train_model_b08dd_00007,2023-03-26_10-11-56,True,,c107a6bee20441dbb6a5e44d10b53fc4,8dd8dad0ce47,1,172.28.0.12,11323,26.5777,26.5777,26.5777,1679825516,0,,1,b08dd_00007,0.76918,0.613641,0.00414228
train_model_b08dd_00008,2023-03-26_10-14-32,True,,60d1f3ccadee48a3b5f4544932af37b2,8dd8dad0ce47,8,172.28.0.12,11499,153.312,18.266,153.312,1679825672,0,,8,b08dd_00008,0.960317,0.115943,0.00409126
train_model_b08dd_00009,2023-03-26_10-15-28,True,,638763e809e048deabac02075d3a02e5,8dd8dad0ce47,2,172.28.0.12,12252,51.7813,24.0609,51.7813,1679825728,0,,2,b08dd_00009,0.919974,0.233204,0.00514102


...

== Status ==
Current time: 2023-03-26 10:12:01 (running for 00:14:02.04)
Memory usage on this node: 2.7/12.7 GiB 
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 8.000: 0.9662698412698413 | Iter 4.000: 0.9626322751322751 | Iter 2.000: 0.951058201058201 | Iter 1.000: 0.8082010582010581
Resources requested: 2.0/2 CPUs, 1.0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.7 GiB objects
Result logdir: /root/ray_results/train_model_2023-03-26_09-57-59
Number of trials: 10/10 (1 PENDING, 1 RUNNING, 8 TERMINATED)
+-------------------------+------------+-------------------+--------------+--------+----------------+------------+-----------+----------------------+
| Trial name              | status     | loc               |   batch_size |     lr |   weight_decay |   val_loss |   val_acc |   training_iteration |
|-------------------------+------------+-------------------+--------------+--------+----------------+------------+-----------+----------------------|
| train_model_b08dd_00008 | RUNNING    | 172.28.0.12

(train_model pid=12252) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(train_model pid=12252)   warnings.warn(
(train_model pid=12252) /usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
(train_model pid=12252)   warnings.warn(msg)


== Status ==
Current time: 2023-03-26 10:14:38 (running for 00:16:39.06)
Memory usage on this node: 2.3/12.7 GiB 
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 8.000: 0.9656084656084656 | Iter 4.000: 0.9636243386243386 | Iter 2.000: 0.9537037037037037 | Iter 1.000: 0.8115079365079365
Resources requested: 2.0/2 CPUs, 1.0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.7 GiB objects
Result logdir: /root/ray_results/train_model_2023-03-26_09-57-59
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+-------------------------+------------+-------------------+--------------+--------+----------------+------------+-----------+----------------------+
| Trial name              | status     | loc               |   batch_size |     lr |   weight_decay |   val_loss |   val_acc |   training_iteration |
|-------------------------+------------+-------------------+--------------+--------+----------------+------------+-----------+----------------------|
| train_model_b08dd_00009 | RUNNING    | 172.28.0.12:12252 |  

2023-03-26 10:15:28,492	INFO tune.py:798 -- Total run time: 1049.03 seconds (1048.98 seconds for the tuning loop).


== Status ==
Current time: 2023-03-26 10:15:28 (running for 00:17:28.99)
Memory usage on this node: 4.1/12.7 GiB 
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 8.000: 0.9656084656084656 | Iter 4.000: 0.9636243386243386 | Iter 2.000: 0.951058201058201 | Iter 1.000: 0.8578042328042328
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/7.41 GiB heap, 0.0/3.7 GiB objects
Result logdir: /root/ray_results/train_model_2023-03-26_09-57-59
Number of trials: 10/10 (10 TERMINATED)
+-------------------------+------------+-------------------+--------------+--------+----------------+------------+-----------+----------------------+
| Trial name              | status     | loc               |   batch_size |     lr |   weight_decay |   val_loss |   val_acc |   training_iteration |
|-------------------------+------------+-------------------+--------------+--------+----------------+------------+-----------+----------------------|
| train_model_b08dd_00000 | TERMINATED | 172.28.0.12:7402  |           64 | 